# Criptografia Aplicada - Laboratorial #4

## Exercício 1

- x³ + x + 1
- x⁴ + x + 1
- x⁴ + x³ + x² 1

### Exercício 1.1

**x³ + x + 1**

R0 = 0111 -> R1 = 1010 -> R2 = 1110 -> R3 = 1001 -> R4 = 0011 -> R5 = 0111 -> R6 = 1110 -> 
-> R7 = 1101 -> R8 = 1010

Como o polinómio é de grau 3, o período máximo esperado é 7, o qual aconteceu.
Podemos concluir que o polinómio é primitivo.

**x⁴ + x + 1**

R0 = 0001 -> R1 = 0011 -> R2 = 0111 -> R3 = 1111 -> R4 = 1110 -> R5 = 1101 -> R6 = 1010 ->
-> R7 = 0101 -> R8 = 1011 -> R9 = 0110 -> R10 = 1100 -> R11 = 1001 -> R12 = 0010 -> 
-> R13 = 0100 -> R14 = 1000

Para este polinómio, também podemos concluir que é primitivo. Isto já que é de grau 4, e tem um periodo igual a 15, o que é esperado se for primitivo.

**x⁴ + x³ + x² 1**

R0 = 0001 -> R1 = 0010 -> R2 = 0101 -> R3 = 1011 -> R4 = 0110 -> R5 = 1100 -> R6 = 1000 -> R7 = 0001

R0 = 0011 -> R1 = 0111 -> R2 = 1110 -> R3 = 1101 -> R4 = 1010 -> R5 = 0100 -> R6 = 1001 -> R7 = 0011

R0 = 0111 -> R1 = 1110 (o estado 1110 já aparece no segundo ciclo mostrado em cima).

Para este caso, podemos concluir que o polinómio **não** é primitivo. Isto já que é de grau 4 e o seu período é de 7.

### Exercício 1.2

O melhor polinómio para o LFSR deve de ser primitivo, o que nos deixa entre e o primeiro e segundo polinómio. No entanto, também sabemos que enquanto maior o período, melhor o polinómio para o LFSR. Logo, o segundo polinómio **x⁴ + x + 1** é o melhor.

### Exercício 1.3 


In [41]:
R.<x> = GF(2)[]
p1 = x^3 + x + 1
p2 = x^4 + x + 1
p3 = x^4 + x^3 + x^2 + 1

p_array = [p1, p2, p3]

for p in p_array:
    print(f"Irredutibilidade de {p}: {'✓ OK' if p.is_irreducible() else '✗ FAIL'}")


Irredutibilidade de x^3 + x + 1: ✓ OK
Irredutibilidade de x^4 + x + 1: ✓ OK
Irredutibilidade de x^4 + x^3 + x^2 + 1: ✗ FAIL


Podemos concluir então que os dois primeiros polínomios são irredutíveis, enquanto o terceiro não é.

Logo, podemos também concluir que são polinómios primitivos.

## Exercício 2

Eis a implementação do RC4 em Python:

In [31]:
import operator

def key_scheduling(key):
    "Inicializa e mistura o estado interno com base na chave (KSA)"
    
    sched = [i for i in range(0, 256)]
    
    i = 0
    for j in range(0, 256):
        i = (i + sched[j] + key[j % len(key)]) % 256
        
        tmp = sched[j]
        sched[j] = sched[i]
        sched[i] = tmp
        
    return sched
    

def stream_generation(sched):
    "Gera bytes do keystream de forma infinita (PRGA)"
    
    i = 0
    j = 0
    while True:
        i = (1 + i) % 256
        j = (sched[i] + j) % 256
        
        tmp = sched[j]
        sched[j] = sched[i]
        sched[i] = tmp
        
        yield sched[(sched[i] + sched[j]) % 256]        


def rc4_transform_file(in_path, out_path, key, encoding='utf-8', chunk_size=65536):
    "Mesma função para cifrar e decifrar arquivos com RC4"
    
    # Normalização da chave: aceita str (hex ou texto) e bytes-like
    if isinstance(key, str):
        ks = key.strip()
        is_hex = (len(ks) % 2 == 0) and all(c in '0123456789abcdefABCDEF' for c in ks)
        if is_hex:
            key_bytes = bytes.fromhex(ks)
        else:
            key_bytes = ks.encode(encoding)
    else:
        key_bytes = bytes(key)
    if not key_bytes:
        raise ValueError("Key must not be empty")

    sched = key_scheduling(list(key_bytes))
    key_stream = stream_generation(sched)

    with open(in_path, 'rb') as fin, open(out_path, 'wb') as fout:
        while True:
            chunk = fin.read(chunk_size)
            if not chunk:
                break
            out = bytearray(len(chunk))
            for i, b in enumerate(chunk):
                out[i] = operator.xor(b, next(key_stream))
            fout.write(out)

Vamos agora gerar uma chave e encriptar o nosso ficheiro chamado **teste.txt**, ubicado no diretório **ficheiros_teste**. O ficheiro encriptado é **teste.txt.enc**

A chave respetiva é **a3c1f28d5b0e4f9a7c21d6e0b58c13f2**

In [32]:
key = "a3c1f28d5b0e4f9a7c21d6e0b58c13f2"
directory = "ficheiros_teste/"
encrypted_file = directory + "teste.txt.enc"
file = directory + "teste.txt"


rc4_transform_file(file, encrypted_file, key)

## Exercício 3

Para este exercício usaremos a linha de comandos.

Para desencriptar usaremos o seguinte:

```bash
openssl enc -rc4 -provider legacy -provider default -d -in ficheiros_teste/teste.txt.enc -out ficheiros_teste/teste_dec_openssl.txt -K a3c1f28d5b0e4f9a7c21d6e0b58c13f2 -nosalt -nopad
```

Como podemos apreciar, o ficheiro gerado é igual ao ficheiro que foi inicialmente encriptado com o script de Python.

Para a segunda parte do exercício, será encriptado o ficheiro **teste.txt** com OpenSSL e desencriptado com o script de Python e ver se de facto as implementações são compatíveis.

Primeiro será executado o comando:

```bash
openssl enc -rc4 -provider legacy -provider default -in ficheiros_teste/teste.txt -out ficheiros_teste/teste_openssl.enc -K a3c1f28d5b0e4f9a7c21d6e0b58c13f2 -nosalt -nopad
```

Agora, vamos ver se conseguimos desencriptá-lo

In [35]:
key = "a3c1f28d5b0e4f9a7c21d6e0b58c13f2"
directory = "ficheiros_teste/"
encrypted_file = directory + "teste_openssl.enc"
decrypted_file = directory + "teste_dec_script.txt"


rc4_transform_file(encrypted_file, decrypted_file, key)

Se verificarmos os conteúdos dos ficheiros correspondentes, concluimos que são compatíveis.

## Exercício 4

Para isto, vamos executar este comando para encriptar:

```bash
openssl enc -chacha20 -in ficheiros_teste/teste.txt -out ficheiros_teste/teste_chacha1.bin -K 1234567890abcdef1234567890abcdef1234567890abcdef1234567890abcdef -iv 000000000001234567890abc -nosalt -nopad
```

Vamos executá-lo de novo para criar outro ficheiro encriptado apartir do mesmo ficheiro:

```bash
openssl enc -chacha20 -in ficheiros_teste/teste.txt -out ficheiros_teste/teste_chacha2.bin -K 1234567890abcdef1234567890abcdef1234567890abcdef1234567890abcdef -iv 000000000001234567890abc -nosalt -nopad
```


Para ver a diferencia de ambos, executamos o seguinte:

```bash
diff ficheiros_teste/teste_chacha1.bin ficheiros_teste/teste_chacha2.bin
```

O último comando quando é executado não retorna nada. É exatamente o comportamento esperado já que o comando **diff** não retorna nada se os ficheiros são idênticos.

O **ChaCha20** é um cifrador **determinístico**. Ou seja, sempre que for usada a mesma chave K e o mesmo Nonce iv, é sempre gerada a mesma keystream.

Como é usado o mesmo "plaintext" e a mesma keystream, é normal e esperado ter o mesmo output.

É por isto que nunca devemos reutilizar a mesma combinação de **chave + nonce**.

## Exercício 5
Stream cipher como RC4 e ChaCha20 (e AES-CTR) produzem ciphertexts que têm exatamente o mesmo tamanho do plaintext, sem padding necessário.

Block Cipher modes como o AES-CBC podem produzir ciphertexts mais longos que aquele do plaintext porque necessitam de padding para preencher o blocos completamente
